In [1]:
import sys
import re
import requests
import time

import praw

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser

sys.path.insert(0, r'C:\Users\HARSH\Documents\chatweb3-backend')
from utils import *
set_api_key()

reddit = praw.Reddit(client_id='vNryBhGWEDDKjF1KiRabfQ', client_secret='VgPhCqfZ707LimGMu4FzAHNSfFdQhw', user_agent='crypto_data')

In [2]:
TOOLS = \
'''
Coinbase
MakerDao
UniSwap
opensea
LidoFinance
ethereum
Ledger
YugaLabs
MyEtherWallet
Stash
CurveFinance
'''

In [3]:
TURN2QUES_TEMPLATE = \
'''
Read the following Reddit posts. For each, please identify the questions the author of the post is trying to have answered, or any questions that a reader of the post who knows nothing about the topic is likely to have. 

Subreddit: r/ethereum
Post title: How find the same/similar smart contract on a tesnet?
Post body: I'm trying to copy an app that's running on the main net. It's using a lot of smart contracts and tokens deployed on the main net. Is there anyway I can find these smart contracts on the goerli test net and not copy and deploy them one by one?
Questions:
How can I find testnet deployments of Ethereum mainnet protocols?
What is the goerli testnet?
How can I deploy an app on a testnet? 
How can I integrate with smart contracts on a testnet?

Subreddit: r/ethereum
Post title: My transaction is stuck for over 2h
Post body: Hi

so I made a transaction on uniswap and its in pending state for over 2h. I can not cancel as 'cancel' button on metamask is not avaliable, I can not speed it up as nothing is happening when I hit speed up button. What are my options?
Questions:
How can I cancel a transaction?
How can I speed up a transaction?
How does canceling a transaction work?
How does "speeding up" a transaction work? 
What can I do about a stuck transaction in metamask? 
What do you do when a transaction is pending for a long time? 

Subreddit: r/CoinBase
Post title: XRP disappeared from wallet. What do I do?
Post body:
Questions:
What happened to my XRP?
Why is my XRP missing from my wallet?
How can I recover my XRP?
What should I do if I can't find my XRP in my wallet? 
What are the possible causes of XRP disappearing from my wallet?

Subreddit: r/{subreddit}
Post title: {title}
Post body: {body}
Questions:'''

class ChatOutputParser(BaseOutputParser):

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "chat_output_parser"

    def parse(self, text: str) -> str:
        """Parse the output of an LLM call."""
        text = text.strip()
        text = text.split('\n')
        return text

llm = OpenAI(
    temperature=0.8, max_tokens=-1,
)
output_parser = ChatOutputParser()
turn2ques_prompt = PromptTemplate(
    input_variables=["subreddit", "body", "title"],
    template=TURN2QUES_TEMPLATE,
    output_parser=output_parser,
)
chain = LLMChain(llm=llm, prompt=turn2ques_prompt, verbose=False)

In [4]:
BLACKLISTED_TITLES = ['Frequently Asked Questions + Weekly Discussion Thread',
                      'Welcome to r/Coinbase! | Help: help.coinbase.com | As a reminder, our only official Coinbase Support presence on Reddit is u/coinbasesupport.']

In [5]:
all_ques = []
for tool in TOOLS.strip().split('\n'):
    try:
        hot_posts = reddit.subreddit(tool).hot(limit=300)
        for post in hot_posts:
            title = post.title
            if title in BLACKLISTED_TITLES: continue
            if title.endswith('?') and len(title.split())>5:
                all_ques.append(max(chain.apply_and_parse([{"subreddit": tool, "title": title, "body": post.selftext}])[0], key=len))
                time.sleep(6)
    except:
        print(f'subreddit "{tool}" not found') 
        pass

subreddit "UniSwap" not found


Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 288020 / min. Contact support@openai.com if you continue to have issues..


subreddit "LidoFinance" not found
subreddit "ethereum" not found
subreddit "Ledger" not found
subreddit "MyEtherWallet" not found
subreddit "Stash" not found
subreddit "CurveFinance" not found


In [36]:
with open("init_questions4_1.txt", 'w') as output:
    for row in all_ques:
        output.write(str(row) + '\n')